<a href="https://colab.research.google.com/github/Yash-1812/ML_DL_Projects/blob/main/CancerPrediction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTING DEPENDENCIES

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Device Configuration

In [ ]:
# Checking CUDA availibility
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
data = load_breast_cancer()
X = data.data
y = data.target

In [ ]:
X

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [ ]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2 , random_state=42)

In [ ]:
len(X_train) , len(X_test) , len(y_train) , len(y_test)

(455, 114, 455, 114)

In [ ]:
X_train = torch.tensor(X_train , dtype = torch.float32)
y_train = torch.tensor(y_train , dtype = torch.float32)

X_test = torch.tensor(X_test , dtype = torch.float32)
y_test = torch.tensor(y_test , dtype = torch.float32)

Neural Network

In [ ]:
class NeuralNet(nn.Module):
  def __init__(self , input_size, hidden_size , output_size):
    super(NeuralNet, self).__init__()
    self.fc1 = nn.Linear(input_size , hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size , output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self , x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out


In [ ]:
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.01
epochs = 10000

In [ ]:
model = NeuralNet(input_size , hidden_size , output_size).to(device)

In [ ]:
# Define loss and optimizer

criteria = nn.BCELoss()
optimizer = optim.Adam(model.parameters() , lr = learning_rate)

Training the Neural Network

In [ ]:
for i in range(epochs):
  model.train()
  optimizer.zero_grad()
  outputs = model(X_train)
  loss = criteria(outputs , y_train.view(-1,1))
  loss.backward()
  optimizer.step()

# Calculate accuracy
with torch.no_grad():
  predicted = outputs.round()
  correct = (predicted == y_train.view(-1,1)).sum().item()
  accuracy = correct / len(y_train)

print(f'Epoch [{i+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}')

Epoch [10000/10000], Loss: 62.8571, Accuracy: 0.3714
